In [1]:
def clawringContentImage(href):
    response = requests.get(href)
    
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        
        res = soup.find('h3', {'id' : 'view_content'})
        content = res.text
        images = res.find_all('img', {'class' : 'lazy content-image'})

        if images != []:
            imageList = [url['data-original'] for url in res.find_all('img', {'class' : 'lazy content-image'})]
        else:
            imageList = []
        return content, imageList
    
    else:
        return np.nan, []    

In [2]:
def parsing_save(re):
#     re, save_path, fieldNames = zip_data
#     print(save_path, fieldNames)
    
    re = BeautifulSoup(re, 'html.parser')
    category = re.find('a', {'class' : 'mw_basic_list_category'}).text
    date = re.find('div', {'class' : 'mw_basic_list_datetime media-no-text'}).text
    view = re.find('div', {'class' : 'mw_basic_list_hit media-no-text'}).text
    title = re.find('h3', {'class' : 'media-list-subject flex'}).text.replace('\n', '')
    rank = str(re.find('img')['src']).split('/')[-1].split('.')[0]
    href = re.find_all('a')[1]['href']
    try:
        writer = re.find('a', {'class' : 'sv_member'}).text.strip()
    except:
        writer = re.find('div', {'mw_basic_list_name media-no-text'}).text
    content, imageList = clawringContentImage(href)

    with open(save_path, 'a', newline = '') as csvfile:
        csvWriter = csv.DictWriter(csvfile, fieldnames = fieldNames)
        
        try:
            csvWriter.writerow({'title' : title,
                                'category' : category,
                                'date' : date,
                                'writer' : writer,
                                'rank' : rank,
                                'view' : view,
                                'href' : href,
                                'content' : content,
                                'imageList' : imageList})      
        except:
            print(href, ': Error')

In [3]:
def clawringHref01(url, pageNum, startPage = 1):
    errorNum = 0
#     path_save_base = '../Data'
#     save_path = os.path.join(path_save_base, path_save)
    if not os.path.isfile(save_path):
        tmp = pd.DataFrame(columns = fieldNames)
        tmp.to_csv(save_path, index = False)
        
        with open(save_path, 'w', newline = '') as csvfile:
            csvWriter = csv.DictWriter(csvfile, fieldnames = fieldNames)
            csvWriter.writeheader()

    for page in tqdm(range(startPage, pageNum + 1)):
        page = str(page)
        response = requests.get(url + page)
        if response.status_code == 200:
            html = response.text
            soup = BeautifulSoup(html, 'html.parser')
            re_list = soup.find_all('div', {"class":"list_wrap flex_spbtw"})
            
            re_list = [str(re) for re in re_list]

#             for i in zip(re_list, [save_path] * len(re_list), [fieldNames] * len(re_list)):
#                 parsing_save(i)
            print(multiprocessing.cpu_count())
    
            with multiprocessing.Pool(multiprocessing.cpu_count() -2) as p:
                p.map(parsing_save, re_list)

#             with multiprocessing.Pool(multiprocessing.cpu_count() -2) as p:
#                 print('1234')
#                 p.starmap(parsing_save, [(x, save_path, fieldNames) for x in re_list])
#                 func = partial(parsing_save, )
#                 p.map(parsing_save, (re_list, [save_path] * len(re_list), [fieldNames] * len(re_list)))
#                 p.map(parsing_save, zip(re_list, [save_path] * len(re_list), [fieldNames] * len(re_list)))

#             p = multiprocessing.Process(target = parsing_save, args = zip(re_list, [save_path] * len(re_list), [fieldNames] * len(re_list)))
#             p.start()
#             p.join()

#             for re in re_list:
#                 parsing_save(re, save_path, fieldNames)
#                 parsing_save(re)